# Segmenting and Clustering Neighborhoods in Toronto

In this Jupyter notebook we will explore the neighberhoods in Toronto.
We first start by importing the necessary libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import folium
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import pgeocode
from geopy.geocoders import Nominatim

In this section we will group the different functions used in this code.

In [3]:
CLIENT_ID = 'MB5ZSSZNHKCSHXYIQCYBKIRCCUAEB5EE2GRK5LVGZOE120US' # your Foursquare ID
CLIENT_SECRET = 'GLBR45QLOFUOGVVOJIQXO4ABYLM3NBPXAA3PNHDAZRNGS5FT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 100 

radius = 500 

We use the library BeautifulSoup in order to scrape the following Wikipedia page : https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


In [4]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 

The result is an html, so we convert it into a dataframe. 
In neighberhoods where there is the same postal code area, they are in the same row seperated with a coma.

In [5]:
neighborhoods = pd.read_html(str(table))[0]
neighborhoods['Neighborhood'] = neighborhoods['Neighborhood'].str.replace('/',',')
neighborhoods = neighborhoods[neighborhoods['Borough'] != 'Not assigned']
neighborhoods.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [8]:
nomi = pgeocode.Nominatim('ca')
postal_code_lat, postal_code_lon = [], [] 

for post in neighborhoods['Postal code']:
    postal_code_lat.append(nomi.query_postal_code(post).latitude)
    postal_code_lon.append(nomi.query_postal_code(post).longitude)

neighborhoods['Latitude'], neighborhoods['Longitude'] = postal_code_lat, postal_code_lon

neighborhoods.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.7545,-79.3300
3,M4A,North York,Victoria Village,43.7276,-79.3148
4,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6555,-79.3626
5,M6A,North York,"Lawrence Manor , Lawrence Heights",43.7223,-79.4504
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6641,-79.3889


In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.
